In [1]:
import os
import sys
import json
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
from torch.utils.data import DataLoader
from parse_dataset import NetworkDataset
from load_models import models
from benchmark import BenchmarkSplitModel

In [2]:
conf = {
    "batch_size": 128,
    "epochs": 10,
    "learning_rate": 0.0001,
    "dpu_model": "dpu_gru",
    "model": "gru",
    "device": "dpu"
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
dataset_path = os.path.join(os.getcwd().replace("model_inference", ""), "datasets")

# load label_dict
json_file = os.path.join(dataset_path, "label_index.json")
with open(json_file, 'r') as file:
    label_dict = json.load(file)

# load train, val and test datasets
train_dataset_file = os.path.join(dataset_path, "train_dataset.pt")
X_train, y_train = torch.load(train_dataset_file)

val_dataset_file = os.path.join(dataset_path, "val_dataset.pt")
X_val, y_val = torch.load(val_dataset_file)

test_dataset_file = os.path.join(dataset_path, "test_dataset.pt")
X_test, y_test = torch.load(test_dataset_file)

if conf["model"] != "mlp" and conf["model"] != "light_mlp":
    X_train, X_val, X_test = X_train.unsqueeze(-1), X_val.unsqueeze(-1), X_test.unsqueeze(-1)
    
print(X_test.shape)
print(y_test.shape)

# create train, val and test datasets
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=conf["batch_size"], shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=conf["batch_size"], shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=conf["batch_size"])

torch.Size([101811, 513, 1])
torch.Size([101811])


In [5]:
dpu_model = models[conf["dpu_model"]]
dpu_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", conf["dpu_model"] + "_model.pth")
benchmark = BenchmarkSplitModel(dpu_model, dpu_path, test_loader, conf["batch_size"], conf["dpu_model"])

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/dpu_gru_model.pth!


In [6]:
benchmark.memory_usage()
benchmark.latency()
benchmark.throughput()
benchmark.cpu_usage()
benchmark.accuracy()

[W1126 11:57:32.097824158 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


In [7]:
file = os.path.join(os.getcwd(), "benhcmarks", conf["device"], conf["dpu_model"] + "_" + conf["device"] + "_benchmark.txt")
with open(file, "w") as f:
    for line in benchmark.results:
        f.writelines(line + "\n")

for line in benchmark.results:
    print(line)

Benchmark - dpu_gru model:

Memory usage (MB):
Avg memory usage: 131.554MB
Peak memory usage: 1924.492MB

Model inference latency on one batch (batch size = 128):
Avg latency: 1160.494ms
Min latency: 1096.902ms
Max latency: 1947.028ms

Model inference throughput (batch size = 128):
Throughput: 113.01 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 58.93 seconds
Average CPU usage: 6.52/8 cores

Model (dpu_gru) inference accuracy (%):
Accuracy: 98.44%



